### A formal language for composing networks

Aka weight tying at a bigger, more complex, more structured scale

<img src="../Images/PoggioModnet.png" alt="Modnet">


I like viewing networks at a higher level. Where instead of considering the connections between neurons we are considering the connections between networks. (although it is really the same thing, as a neuron is just some function).

<img src="../Images/PoggioModnetmat.png" alt="Modnetmat">

$$
\begin{bmatrix} 0 & A & 0 \\ 0 & 0 & B \\ C & 0 & D \end{bmatrix} \left[ \begin{array}{c} x \\ y \\ z \end{array} \right] =  \left[ \begin{array}{c} Ay \\ Bz \\ Cx + Dz \end{array} \right] \rightarrow \left[ \begin{array}{c} x \\ y \\ z \end{array} \right]\\\\
$$
$$
x_{t+1} = A y_t \\
y_{t+1} = B z_t \\
z_{t+1} = C x_t + D  z_t
$$

So all we have to do is replace our non-linearity in $\sigma(\phi \cdot W + b)$ with a neural network, $NN(\phi \cdot W + b)$? No, it's not quite that simple... As we want different non-linearities for different states.
$$
\begin{bmatrix} 0 & A & 0 \\ 0 & 0 & B \\ C & 0 & D \end{bmatrix} \left[ \begin{array}{c} x \\ y \\ z \end{array} \right] =  \left[ \begin{array}{c} A(y) \\ B(z) \\ C(x) + D(z) \end{array} \right] \rightarrow \left[ \begin{array}{c} x \\ y \\ z \end{array} \right]\\
$$
Which would be equivalent to 
$$
x_{t+1} = A(y_t) \\
y_{t+1} = B(z_t) \\
z_{t+1} = C(x_t) + D(z_t)
$$

Where;
* x,y,z are tensors. 
* A,B,C,D are neural networks. E.g. A = a 2x5x3 layer feedforward neural net. B = a 10 neuron RNN. ...

### Higher levels

I would then want to be able to say

$$
E = \begin{bmatrix} 0 & A & 0 \\ 0 & 0 & B \\ C & 0 & D \end{bmatrix} \\
F = \begin{bmatrix} A & B & B \\ 0 & 0 & C \\ D & D & 0 \end{bmatrix} \\
G = \begin{bmatrix} 0 & A & 0 \\ 0 & C & B \\ 0 & 0 & D \end{bmatrix} \\
H = \begin{bmatrix} 0 & 0 & D \\ C & 0 & B \\ 0 & A & 0 \end{bmatrix} \\
$$
and finally
$$
I = \begin{bmatrix} E & F & 0 \\ A & 0 & B \\ C & H & D \end{bmatrix} \\
$$
on so on and so forth, for as many higher 'levels' as I wanted.

### Issues and fundamental questions;
* C(x) + D(z) can only be evaluated properly if;
    * the inputs are the right shape, f(x) takes 5x1 but recieves 10x3...
    * the outputs are the right shape. f(x) returns 2x1 and g(y) returns 3x5. how can these be composed together?
        * We could make all nets input/output the same shape?
        * We could create a basis type that all our nets can recieve. Anything else gets (un)raveled?
* What would be the basic building block?
    * the linear neuron? probably not
    * Also these need to be able to proparage forward in time for RNNs.
* So normal matnul would be a special case of this? Where A(x) = A*x
    
    
### Interestingly we could build;
* traditional activation functions from a network of less complex neurons. Thus a universal net?
* a RNN with neurons that are RNNs themselves, thus giving time-dependent activity, or memory.
* So pretty much any neural network architecture is a special case of this.

# Representation?? (universal??)

### Conv net
What would a conv net look like, when composed from simpler parts?/a meta language?

##### Typical coding
```python
o_conv0 = ConvL1(image)
o_conv1 = ConvL2(o_conv0)
o_conv2 = ConvL3(o_conv1)
o_conv3 = ConvL4(o_conv2)
o_conv4 = ConvL5(o_conv3)
o_conv5 = ConvL6(o_conv4)
o_fc0 = FC1(o_conv5)
o_fc1 = FC2(o_fc0)
o_fc3 = FC3(o_fc1)
y = softmax(o_fc3)
```
Another way of looking at it?
$$
\begin{bmatrix} 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
ConvL1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & ConvL2 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & ConvL3 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & ConvL4 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & ConvL5 & 0 & 0 & 0 & 0\\  
0 & 0 & 0 & 0 & 0 & FC1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & FC2 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & FC3 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & Softmax \\
\end{bmatrix}  \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\  \end{array} \right] = \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\  \end{array} \right]
$$

But needs to be evaulated sequentially, or iterated over.

##### 1D
Image we have a kernel/filter/neuron A in a conv layer. Then,
5x7 x 7x1
$$
\begin{bmatrix} 
. & A & . & 0 & 0 & 0 & 0 \\ 
0 & . & A & . & 0 & 0 & 0 \\ 
0 & 0 & . & A & . & 0 & 0 \\ 
0 & 0 & 0 & . & A & . & 0 \\ 
0 & 0 & 0 & 0 & . & A & . \\ 
\end{bmatrix}  \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6\\ x_7\\ \end{array} \right] = \left[ \begin{array}{c} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5  \\ \end{array} \right] 
$$

or with stride = 2

$$
\begin{bmatrix} 
. & A & . & 0 & 0 & 0 & 0 \\ 
0 & 0 & . & A & 0 & 0 & 0 \\ 
0 & 0 & 0 & 0 & . & A & . \\ 
\end{bmatrix}  \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6\\ x_7\\ \end{array} \right] = \left[ \begin{array}{c} y_1 \\ y_2 \\ y_3 \\ \end{array} \right] 
$$
##### 2D
Hmm, so we require a matrix of 1D higher than the dimensions we are convolving over.
5x15x7 x 7x7 -> 5x(15x7) -> 
$$
\begin{bmatrix} 
. & . & . & 0 & 0 & 0 & 0 \\ 
. & A & . & 0 & 0 & 0 & 0 \\ 
. & . & . & 0 & 0 & 0 & 0 \\ 
0 & . & . & . & 0 & 0 & 0 \\ 
0 & . & A & . & 0 & 0 & 0 \\
0 & . & . & . & 0 & 0 & 0 \\ 
0 & 0 & . & . & . & 0 & 0 \\ 
0 & 0 & . & A & . & 0 & 0 \\ 
0 & 0 & . & . & . & 0 & 0 \\ 
0 & 0 & 0 & . & . & . & 0 \\ 
0 & 0 & 0 & . & A & . & 0 \\ 
0 & 0 & 0 & . & . & . & 0 \\ 
0 & 0 & 0 & 0 & . & . & . \\ 
0 & 0 & 0 & 0 & . & A & . \\ 
0 & 0 & 0 & 0 & . & . & . \\ 
\end{bmatrix} 
\begin{bmatrix}
x_{1,1} & x_{1,2} & x_{1,3} & x_{1,4} & x_{1,5} & x_{1,6} & x_{1,7} \\
x_{2,1} & x_{2,2} & x_{2,3} & x_{2,4} & x_{2,5} & x_{2,6} & x_{2,7} \\
x_{3,1} & x_{3,2} & x_{3,3} & x_{3,4} & x_{3,5} & x_{3,6} & x_{3,7} \\
x_{4,1} & x_{4,2} & x_{4,3} & x_{4,4} & x_{4,5} & x_{4,6} & x_{4,7} \\
x_{5,1} & x_{5,2} & x_{5,3} & x_{5,4} & x_{5,5} & x_{5,6} & x_{5,7} \\
x_{6,1} & x_{6,2} & x_{6,3} & x_{6,4} & x_{6,5} & x_{6,6} & x_{6,7} \\
x_{7,1} & x_{7,2} & x_{7,3} & x_{7,4} & x_{7,5} & x_{7,6} & x_{7,7} \\
\end{bmatrix}
= \left[ \begin{array}{c} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \\ y_6\\ y_7\\ y_8 \\ y_9 \\ y_{10} \\ y_{11} \\ y_{12} \\ y_{13}\\ y_{14}\\ y_{15} \\ \end{array} \right] \rightarrow  \left[ \begin{array}{c} y_1 + y_2 + y_3 \\ y_4 + y_5 + y_6\\ y_7 + y_8 + y_9 \\ y_{10} + y_{11} + y_{12} \\ y_{13} + y_{14} + y_{15} \\ \end{array} \right]
$$

##### Multi-layer??
What about a multilayer conv net?? How can I represent that with this language?



##### R-CNN?

Def $B: \vec{x} \rightarrow \vec{z}$ such that $\vec{z} = A \ast \vec{x}$

$$
RCNN = \begin{bmatrix} 0 & B \\ B & 0 \\ \end{bmatrix}
$$
so x = B(y) and y = B(x), a recurrent net (kinda - without states).
$$
\begin{bmatrix} 
0&0&0&0&0&0&0& . & . & . & 0 & 0 & 0 & 0 \\ 
0&0&0&0&0&0&0& . & A & . & 0 & 0 & 0 & 0 \\ 
0&0&0&0&0&0&0& . & . & . & 0 & 0 & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & . & . & . & 0 & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & . & A & . & 0 & 0 & 0 \\
0&0&0&0&0&0&0& 0 & . & . & . & 0 & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & . & . & . & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & . & A & . & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & . & . & . & 0 & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & . & . & . & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & . & A & . & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & . & . & . & 0 \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & 0 & . & . & . \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & 0 & . & A & . \\ 
0&0&0&0&0&0&0& 0 & 0 & 0 & 0 & . & . & . \\ 
. & . & . & 0 & 0 & 0 & 0 &0&0&0&0&0&0&0\\ 
. & A & . & 0 & 0 & 0 & 0 &0&0&0&0&0&0&0\\ 
. & . & . & 0 & 0 & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & . & . & . & 0 & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & . & A & . & 0 & 0 & 0 &0&0&0&0&0&0&0\\
0 & . & . & . & 0 & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & . & . & . & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & . & A & . & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & . & . & . & 0 & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & . & . & . & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & . & A & . & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & . & . & . & 0 &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & 0 & . & . & . &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & 0 & . & A & . &0&0&0&0&0&0&0\\ 
0 & 0 & 0 & 0 & . & . & . &0&0&0&0&0&0&0\\
\end{bmatrix}
\left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6\\ x_7\\ y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \\ y_6\\ y_7\\\end{array} \right] = \left[ \begin{array}{c} z_1 \\ z_2 \\ z_3 \\ z_4 \\ z_5 \\ z_6\\ z_7\\ z_8 \\ z_9 \\ z_{10} \\ z_{11} \\ z_{12} \\ z_{13}\\ z_{14}\\ z_{15}\\ z_{16} \\ z_{17} \\ z_{18} \\ z_{19} \\ z_{20} \\ z_{21}\\ z_{22}\\ z_{23} \\ z_{24} \\ z_{25} \\ z_{26} \\ z_{27} \\ z_{28}\\ z_{29}\\ z_{30} \\ \end{array} \right] \rightarrow Oh...
$$


### RNNs

$$
\begin{bmatrix} 
RNN & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & RNN & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & RNN & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & RNN & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & RNN & 0 & 0 & 0 & 0\\  
0 & 0 & 0 & 0 & 0 & RNN & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & RNN & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & RNN & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & RNN\\
\end{bmatrix}  \left[ \begin{array}{c} \vec{x_1} \\ \vec{x_2} \\ \vec{x_3} \\ \vec{x_4} \\ \vec{x_5} \\ \vec{x_6} \\ \vec{x_7} \\ \vec{x_8} \\ \vec{x_9} \\  \end{array} \right] 
$$
where x_i = a vector of all states
argh... this isnt functional... how would this work?
### Clockwork RNN

C clock work RNN with 

$$
\begin{bmatrix} 
RNN_{t=1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & RNN_{t=2} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
RNN & 0 & RNN_{t=3} & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & RN_{t=4} & 0 & 0 & 0 & 0 & 0\\
0 & 0 & RNN & 0 & RNN & 0 & 0 & 0 & 0\\  
0 & 0 & 0 & 0 & 0 & RNN & 0 & 0 & 0\\
0 & 0 & 0 & 0 & RNN & 0 & RNN & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & RNN & 0\\
0 & 0 & 0 & 0 & 0 & 0 & RNN & 0 & RNN\\
\end{bmatrix}  \left[ \begin{array}{c} \vec{x_1} \\ \vec{x_2} \\ \vec{x_3} \\ \vec{x_4} \\ \vec{x_5} \\ \vec{x_6} \\ \vec{x_7} \\ \vec{x_8} \\ \vec{x_9} \\  \end{array} \right] 
$$


### Resnet

$$
\begin{bmatrix} 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
ConvL1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & ConvL2 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
ConvL1 & 0 & ConvL3 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & ConvL4 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & ConvL3 & 0 & ConvL5 & 0 & 0 & 0 & 0\\  
0 & 0 & 0 & 0 & 0 & FC1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & FC2 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & FC3 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & Softmax \\
\end{bmatrix}  \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\  \end{array} \right] = \left[ \begin{array}{c} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\  \end{array} \right]
$$
He el at..
### other nets? ???